# Lesson 1

## Linear Classification and Logistic Regression

In [2]:
import pandas as pd

In [64]:
import numpy as np

In [3]:
#read the dataset
df = pd.read_csv('https://query.data.world/s/wh6j7rxy2hvrn4ml75ci62apk5hgae')

C:\Users\USER\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
df.head()

,country,year,country_code,record,crop_land,grazing_land,forest_land,fishing_ground,built_up_land,carbon,total,QScore
0,Armenia,1992,1,AreaPerCap,0.140292,0.199546,0.097188051,0.036888,0.029320,0.000000,5.032351e-01,3A
1,Armenia,1992,1,AreaTotHA,483000.000000,687000.000000,334600,127000.000000,100943.000800,0.000000,1.732543e+06,3A
2,Armenia,1992,1,BiocapPerCap,0.159804,0.135261,0.084003213,0.013742,0.033398,0.000000,4.262086e-01,3A
3,Armenia,1992,1,BiocapTotGHA,550176.242700,465677.972200,289207.1078,47311.551720,114982.279300,0.000000,1.467355e+06,3A
4,Armenia,1992,1,EFConsPerCap,0.387510,0.189462,1.26E-06,0.004165,0.033398,1.114093,1.728629e+00,3A


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72186 entries, 0 to 72185
Data columns (total 12 columns):
country           72186 non-null object
year              72186 non-null int64
country_code      72186 non-null int64
record            72186 non-null object
crop_land         51714 non-null float64
grazing_land      51714 non-null float64
forest_land       51714 non-null object
fishing_ground    51713 non-null float64
built_up_land     51713 non-null float64
carbon            51713 non-null float64
total             72177 non-null float64
QScore            72185 non-null object
dtypes: float64(6), int64(2), object(4)
memory usage: 6.6+ MB


In [6]:
#Check the target variable classes
df['QScore'].value_counts()

3A    51481
2A    10576
2B    10096
1B       16
1A       16
Name: QScore, dtype: int64

In [7]:
#Check for missing values
df.isna().sum()

country               0
year                  0
country_code          0
record                0
crop_land         20472
grazing_land      20472
forest_land       20472
fishing_ground    20473
built_up_land     20473
carbon            20473
total                 9
QScore                1
dtype: int64

In [8]:
#for simplicity, we will drop the rows with missing values.
df = df.dropna()

In [9]:
#Check for missing values
df.isna().sum()

country           0
year              0
country_code      0
record            0
crop_land         0
grazing_land      0
forest_land       0
fishing_ground    0
built_up_land     0
carbon            0
total             0
QScore            0
dtype: int64

In [10]:
#Check the target variable classes
df['QScore'].value_counts()

3A    51473
2A      224
1A       16
Name: QScore, dtype: int64

In [11]:
#An obvious change in our target variable after removing the missing values is that there are only three classes left 
#and from the distribution of the 3 classes, we can see that there is an obvious imbalance between the classes. 
#There are methods that can be applied to handle this imbalance such as oversampling and undersampling.
#Oversampling involves increasing the number of instances in the class with fewer instances while undersampling 
#involves reducing the data points in the class with more instances.


In [12]:
#For now, we will convert this to a binary classification problem by combining class '2A' and '1A'.
df['QScore'] = df['QScore'].replace(['1A'], '2A')

In [13]:
df.QScore.value_counts()

3A    51473
2A      240
Name: QScore, dtype: int64

In [14]:
df_2A = df[df.QScore=='2A'] #new dataframe for 2A
df_3A = df[df.QScore=='3A'].sample(350) #new dataframe form 3A
data_df = df_2A.append(df_3A) #Join both dataframe together

In [15]:
import sklearn.utils
data_df = sklearn.utils.shuffle(data_df)


In [16]:
data_df = data_df.reset_index(drop=True) #reset index of dataframe


In [17]:
print(data_df.shape)
data_df.QScore.value_counts()

(590, 12)


3A    350
2A    240
Name: QScore, dtype: int64

In [18]:
#more preprocessing
data_df = data_df.drop(columns=['country_code', 'country', 'year'])
X = data_df.drop(columns='QScore')
y = data_df['QScore']

In [19]:
#split the data into training and testing sets
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
y_train.value_counts()

3A    243
2A    170
Name: QScore, dtype: int64

In [20]:
#There is still an imbalance in the class distribution. For this, we use SMOTE only on the training data to handle this.


In [21]:
#encode categorical variable
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
x_train.record = encoder.fit_transform(x_train.record)
x_test.record = encoder.transform(x_test.record)

C:\Users\USER\Anaconda3\lib\site-packages\pandas\core\generic.py:5096: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [22]:
import imblearn

In [23]:
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=1)
x_train_balanced, y_balanced = smote.fit_sample(x_train, y_train)

In [25]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [52]:
normalised_train_df = scaler.fit_transform(x_train_balanced.drop(columns=['record']))
normalised_train_df = pd.DataFrame(normalised_train_df, columns=x_train_balanced.drop(columns=['record']).columns)
normalised_train_df['record'] = x_train_balanced['record']


In [53]:
x_test = x_test.reset_index(drop=True)
normalised_test_df = scaler.transform(x_test.drop(columns=['record']))
normalised_test_df = pd.DataFrame(normalised_test_df, columns=x_test.drop(columns=['record']).columns)
normalised_test_df['record'] = x_test['record']

In [54]:
#Logistic Regression
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression()

In [55]:
log_reg.fit(normalised_train_df, y_balanced)

LogisticRegression()

In [56]:
new_predictions = log_reg.predict(normalised_test_df)

# Lesson 2

## Measuring Classification Performance

In [32]:
from sklearn.metrics import recall_score, accuracy_score, precision_score, f1_score, confusion_matrix

### Accuracy

In [57]:
accuracy = accuracy_score(y_true=y_test, y_pred=new_predictions)
print('Accuracy: {}'.format(round(accuracy*100), 2)) #prints 53.0

Accuracy: 63.0


### Precision score

In [58]:
precision = precision_score(y_true=y_test, y_pred=new_predictions, pos_label='2A')
print('Precision: {}'.format(round(precision*100), 2)) #prints 41.0

Precision: 92.0


### Recall Score

In [59]:
recall = recall_score(y_true=y_test, y_pred=new_predictions, pos_label='2A')
print('Recall: {}'.format(round(recall*100), 2)) #prints 51.0 

Recall: 64.0


### Confusion matrix

In [60]:
cnf_mat = confusion_matrix(y_true=y_test, y_pred=new_predictions, labels=['2A', '3A'])
cnf_mat

array([[54, 31],
       [ 5,  7]], dtype=int64)

### Cross Validation

In [35]:
from sklearn.model_selection import cross_val_score

In [36]:
scores = cross_val_score(log_reg, normalised_train_df, y_balanced, cv=5, scoring='f1_macro')
scores

array([0.48787625, 0.5025641 , 0.54245283, 0.45267753, 0.52572279])

### F1 - Score 

In [37]:
f1 = f1_score(y_true=y_test, y_pred=new_predictions, pos_label='2A')
print('F1: {}'.format(round(f1*100), 2)) #prints 45.0

F1: 41.0


### K-Fold Cross validation 

In [38]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=5)

In [39]:
kf.split(normalised_train_df)
f1_scores = []

In [40]:
for train_index, test_index in kf.split(normalised_train_df):
    x_train, x_test = normalised_train_df.iloc[train_index],normalised_train_df.iloc[test_index]
    y_train, y_test = y_balanced[train_index],y_balanced[test_index]
    
    model = LogisticRegression().fit(x_train, y_train)
  #save result to list
    f1_scores.append(f1_score(y_true=y_test, y_pred=model.predict(x_test), pos_label='2A')*100)

In [41]:
f1_scores

[44.85981308411215,
 59.09090909090909,
 51.19999999999999,
 65.67164179104478,
 0.0]

### Leave One Out Cross Validation (LOOCV)

In [42]:
from sklearn.model_selection import LeaveOneOut
loo = LeaveOneOut()
scores = cross_val_score(LogisticRegression(), normalised_train_df, y_balanced, cv=loo,  scoring='f1_macro')
average_score = scores.mean() * 100

In [44]:
average_score

47.94238683127572

### Stratified K - Fold Cross Validation

In [61]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
f1_scores = []

In [65]:
#run for every split
for train_index, test_index in skf.split(normalised_train_df, y_balanced):
    x_train, x_test = np.array(normalised_train_df)[train_index], np.array(normalised_train_df)[test_index]
    y_train, y_test  = y_balanced[train_index], y_balanced[test_index]
    model = LogisticRegression().fit(x_train, y_train)
  #save result to list
    f1_scores.append(f1_score(y_true=y_test, y_pred=model.predict(x_test), pos_label='2A'))

In [66]:
f1_scores

[0.5283018867924528,
 0.576923076923077,
 0.594059405940594,
 0.4421052631578947,
 0.37777777777777777]